# IMPORTS AND CONSTANTS

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler as Scaler
from functools import partial
from pycm import ConfusionMatrix

dataset_filepath = "gs://ecb-fsf-hackathon-base-data/webscrap_train.csv"
index_header = 'url'
non_empty_columns = ("Predicted_COICOP_5_Category", "Predicted_COICOP_4_Category", "Predicted_COICOP_3_Category", "Predicted_COICOP_2_Category",
                     "Coicop5_Final", "Coicop4_Final", "Coicop3_Final", "Coicop2_Final")

predicted_target_pairs = [('Predicted_COICOP_5_Category', 'Coicop5_Final'),
                          ('Predicted_COICOP_4_Category', 'Coicop4_Final'),
                          ('Predicted_COICOP_3_Category', 'Coicop3_Final'),
                          ('Predicted_COICOP_2_Category', 'Coicop2_Final'),
                         ]
level_weights = {'Coicop2_Final': 0.05,
                 'Coicop3_Final': 0.15,
                 'Coicop4_Final': 0.4,
                 'Coicop5_Final': 0.4}


# FUNCTION DEFINITIONS

In [14]:
def clean_empty_cells(df: pd.DataFrame) -> pd.DataFrame:
    """Replace all empty cells with pd.np.nan"""
    return df.applymap(lambda x: pd.np.nan if isinstance(x, str)
                       and x.lower().strip() in ['nan', ''] else x)

def drop_duplicate_rows(df: pd.DataFrame, index) -> pd.DataFrame:
    """Drop duplicate rows by index"""
    return df.drop_duplicates(subset=index)

def drop_empty_rows(df: pd.DataFrame,
                  required_rows: tuple=()) -> pd.DataFrame:
    """Remove row without the required data points""" 
    df = df.dropna(subset=non_empty_columns, how='any')
    return df

def calc_score_for_level(confusion_matrix: ConfusionMatrix,
                         balance: bool) -> float:
    """Unified metric for a single level multiclass classifier
    
    :param balance: If True, f1 score is normalized by class frequency
    """
    n_classes = confusion_matrix.classes.__len__()
    scores = []
    if not balance:
        score = confusion_matrix.F1_Macro
    else:
        for cls in confusion_matrix.classes:
            population = confusion_matrix.POP[cls]
            score = confusion_matrix.F1[cls]
            if balance:
                cls_count = confusion_matrix.P[cls]
                if cls_count == 0:
                    score = 0.
                else:
                    balance_ratio = population / (n_classes * cls_count)
                    score *= balance_ratio
            scores.append(score)
        scores = np.expand_dims(np.array(scores), 1)
        scores = Scaler().fit_transform(scores)
        score = scores.ravel().mean()
    return score

# READ THE DATASET

In [15]:
dataset_cleaned_up = False
df = pd.read_csv(dataset_filepath, dtype=str)
df.head(2)

,Unnamed: 0,category,url,product_name,product_description,product_id_store,product_id,volume,qty,unit,...,reduction,product,Coicop5_Final,Coicop4_Final,Coicop3_Final,Coicop2_Final,Predicted_COICOP_5_Category,Predicted_COICOP_4_Category,Predicted_COICOP_3_Category,Predicted_COICOP_2_Category
0,24345,"Kaffee, Tee & Kakao Tee Kräuter- & Früchtetee ...",https://shop.rewe.de/p/kraeutertee-vitaknall/K...,Kräutertee Vitaknall,Dieser leckere Kräuter-Tee mit Fliederbeer-Man...,p/kraeutertee-vitaknall/K4M1SSOL,742c290660c87a47fd35930ef9a7b47f,250 G (100 G = 3 €),250,gram,...,False,https://shop.rewe.de/p/kraeutertee-vitaknall/K...,01212,0121,012,01,NaN,NaN,NaN,NaN
1,21651,Lebensmittel Wurst-/Fleischwaren,https://www.edeka24.de/Lebensmittel/Wurst-Flei...,Meica Ragout Fin,Zubereitet mit zartem Geflügel- und Kalbfleisc...,04693f48ee73812a14551941626657e8,55464b6d5ce42feceb8fccee9e84dec4,Inhalt: 400 g,400,gram,...,False,https://www.edeka24.de/Lebensmittel/Wurst-Flei...,01128,0112,011,01,NaN,NaN,NaN,NaN


# CLEANUP DATASET

In [16]:
if not dataset_cleaned_up:
    cleanup_pipeline = [clean_empty_cells,
                        partial(drop_duplicate_rows, index=index_header),
                        partial(drop_empty_rows, required_rows=non_empty_columns)]
    for op in cleanup_pipeline:
        df = op(df)
    dataset_cleaned_up = True

# COLLECT PERFORMANCE STATS

In [17]:
confusion_matrices = {actual: ConfusionMatrix(actual_vector=df[actual].values,
                                              predict_vector=df[predicted].values)
                      for predicted, actual in predicted_target_pairs}

pycmVectorError: Input vectors are empty

# CALCULATE SCORES

In [18]:
score_levels = {level: calc_score_for_level(cm, balance=True) for  level, cm in confusion_matrices.items()}
final_score = sum([level_weights[level] * value for level, value in score_levels.items()])
print("The final score is {final_score}".format(final_score=final_score))

NameError: name 'confusion_matrices' is not defined